### Use Machine Learning to solve Differential Equations

Physics-informed neural network is presented in more details in this paper: "DeepXDE: A deep learning library for solving differential equations" (https://arxiv.org/abs/1907.04502).

Solve the following ordinary differential equations for $u=u(x)$ with $0\le x\le 1$:
$$
\frac{d^2 u}{d x^2} = - 4\pi^2 \sin(2\pi x), \quad 0 \le x \le 1
$$
with Dirichlet boundary conditions:
$$
u(0) = 0, \quad u(1) = 0
$$

We can solve out the exact solution by hand:
$$
u(x) = \sin(2\pi x)
$$

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
import torch
import torch.nn.functional as F

# feedforward neural network
class FNN(torch.nn.Module):
    def __init__(self, num=32):
        super(FNN, self).__init__()
        self.l1 = torch.nn.Linear(1, num)
        self.l2 = torch.nn.Linear(num, num)
        self.l3 = torch.nn.Linear(num, num)
        self.l4 = torch.nn.Linear(num, 1)

    def forward(self, x):
        z = F.tanh(self.l1(x))
        z = F.tanh(self.l2(z))
        z = F.tanh(self.l3(z))
        return self.l4(z)
    
    # compute the first-order derivative of u with respect to x    
    def dx(self, x):
        u = self.forward(x)
        vec_one = torch.ones(x.shape)
        ux = torch.autograd.grad(u, x, grad_outputs = vec_one, create_graph = True)[0]
        return ux

    # compute the second-order derivative of u with respect to x        
    def dxx(self, x):
        u = self.forward(x)
        vec_one = torch.ones(x.shape)
        ux = torch.autograd.grad(u, x, grad_outputs = vec_one, create_graph = True)[0]
        uxx = torch.autograd.grad(ux, x, grad_outputs = vec_one, create_graph = True)[0]
        return uxx


In [ ]:
# solve u_xx = -4*pi^2*sin(x)
def source(x):
    return - 4 * math.pi**2 * torch.sin(2*math.pi*x)

# exact solution u = sin(2*pi*x)
def exact(x):
    return torch.sin(2*math.pi*x)

def dirichlet_bc(x_bc):
    return exact(x_bc)

# computational domain 0 < x < 1
Nx = 100
x = torch.linspace(0., 1., Nx, requires_grad=True)
x = torch.reshape(x, (Nx,1))

# boundary points
x_bd = torch.tensor([[0.], [1.]], requires_grad=True)

# neural network
model = FNN()

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

total_epoch = 10000

for epoch in range(total_epoch):
    
    # set grad to be zero in the optimizer
    optimizer.zero_grad()
    
    # loss function
    loss = criterion(model.dxx(x), source(x)) + criterion(model(x_bd), dirichlet_bc(x_bd))

    loss.backward()
    optimizer.step()

    if epoch%1000 == 0:
        print(epoch, loss.item())    
        
# plot the solution from neural network and the exact solution
plt.figure()
plt.plot(x.detach(), model(x).detach(), 'o', markersize=2, label='neural network solution')
plt.plot(x.detach(), exact(x).detach(), '-', label='exact solution')
plt.legend()

plt.show()

## DeepXDE: a library for scientific machine learning and physics-informed learning

https://github.com/lululxvi/deepxde

In [ ]:
%env DDE_BACKEND=pytorch

### second-order ODE

In [ ]:
import deepxde as dde
import numpy as np

# solve second-order ODE for y = y(t)
# 
# y'' - 10 * y' + 9 * y = 5 * t
# 
# with initial condition
# 
# y(0) = -1
# y'(0) = 2

def ode(t, y):
    dy_dt = dde.grad.jacobian(y, t)
    d2y_dt2 = dde.grad.hessian(y, t)
    return d2y_dt2 - 10 * dy_dt + 9 * y - 5 * t


def func(t):
    return 50 / 81 + t * 5 / 9 - 2 * np.exp(t) + (31 / 81) * np.exp(9 * t)


geom = dde.geometry.TimeDomain(0, 0.25)


def boundary_l(t, on_initial):
    return on_initial and np.isclose(t[0], 0)


def bc_func1(inputs, outputs, X):
    return outputs + 1


def bc_func2(inputs, outputs, X):
    return dde.grad.jacobian(outputs, inputs, i=0, j=None) - 2

# y(0) = -1
ic1 = dde.icbc.IC(geom, lambda x: -1, lambda _, on_initial: on_initial)
# y'(0) = 2
ic2 = dde.icbc.OperatorBC(geom, bc_func2, boundary_l)

# number 16 is the number of training residual points sampled inside the domain
# number 2 is the number of training points sampled on the boundary
data = dde.data.TimePDE(geom, ode, [ic1, ic2], 16, 2, solution=func, num_test=500)
layer_size = [1] + [50] * 3 + [1]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.nn.FNN(layer_size, activation, initializer)

model = dde.Model(data, net)
model.compile("adam", lr=0.001, metrics=["l2 relative error"], loss_weights=[0.01, 1, 1])
losshistory, train_state = model.train(iterations=10000)

dde.saveplot(losshistory, train_state, issave=True, isplot=True)

### ODE system

In [ ]:
import deepxde as dde
import numpy as np


def ode_system(x, y):
    """ODE system.
    dy1/dx = y2
    dy2/dx = -y1
    """
    # the first dimension of y tensor is the training points
    # the second dimension o y tensor is the componets of the solution
    y1, y2 = y[:, 0:1], y[:, 1:]
    dy1_x = dde.grad.jacobian(y, x, i=0)
    dy2_x = dde.grad.jacobian(y, x, i=1)

    return [dy1_x - y2, dy2_x + y1]


def boundary(_, on_initial):
    return on_initial


def func(x):
    """
    y1 = sin(x)
    y2 = cos(x)
    """
    return np.hstack((np.sin(x), np.cos(x)))

# 0 < x < 10
geom = dde.geometry.TimeDomain(0, 10)
# y1(0) = 0
ic1 = dde.icbc.IC(geom, lambda x: 0, boundary, component=0)
# y2(0) = 1
ic2 = dde.icbc.IC(geom, lambda x: 1, boundary, component=1)
data = dde.data.PDE(geom, ode_system, [ic1, ic2], 35, 2, solution=func, num_test=100)

layer_size = [1] + [50] * 3 + [2]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.nn.FNN(layer_size, activation, initializer)

model = dde.Model(data, net)
model.compile("adam", lr=0.001, metrics=["l2 relative error"])
losshistory, train_state = model.train(iterations=20000)

dde.saveplot(losshistory, train_state, issave=True, isplot=True)

### Poisson equation

In [ ]:
import deepxde as dde
import numpy as np

# solve Poisson equation in 1D
# 
# u_xx = 2, for -1 < x < 1
# 
# with Dirichlet boundary condition on the left boundary
# u(-1) = 0
# and Neumann boundary condition on the right boundary
# u_x(1) = 4
# 
# The exact solution is
# 
# u(x) = (x + 1)^2
# 
def pde(x, y):
    dy_xx = dde.grad.hessian(y, x)
    return dy_xx - 2


def boundary_l(x, on_boundary):
    return on_boundary and np.isclose(x[0], -1)


def boundary_r(x, on_boundary):
    return on_boundary and np.isclose(x[0], 1)


def func(x):
    return (x + 1) ** 2


geom = dde.geometry.Interval(-1, 1)
bc_l = dde.icbc.DirichletBC(geom, func, boundary_l)
bc_r = dde.icbc.NeumannBC(geom, lambda X: 2 * (X + 1), boundary_r)
data = dde.data.PDE(geom, pde, [bc_l, bc_r], 16, 2, solution=func, num_test=100)

layer_size = [1] + [50] * 3 + [1]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.nn.FNN(layer_size, activation, initializer)

model = dde.Model(data, net)
model.compile("adam", lr=0.001, metrics=["l2 relative error"])
losshistory, train_state = model.train(iterations=10000)

dde.saveplot(losshistory, train_state, issave=True, isplot=True)

### Heat equation

In [ ]:
import deepxde as dde
import numpy as np

# solve heat equation
# u_t = a * u_xx, 0 < x < 1 and 0 < t < 1
# 
# a is thermal diffusivity constant
# 
# with Dirichlet boundary condition
# u(0,t) = 0 and u(1,t) = 0
# 
# initial condition
# u(x,0) = sin(n*pi*x/L)
# L is the length of bar and n is the frequency of the sinusoidal initial conditions

def heat_eq_exact_solution(x, t):
    """Returns the exact solution for a given x and t (for sinusoidal initial conditions).

    Parameters
    ----------
    x : np.ndarray
    t : np.ndarray
    """
    return np.exp(-(n**2 * np.pi**2 * a * t) / (L**2)) * np.sin(n * np.pi * x / L)


def gen_exact_solution():
    """Generates exact solution for the heat equation for the given values of x and t."""
    # Number of points in each dimension:
    x_dim, t_dim = (256, 201)

    # Bounds of 'x' and 't':
    x_min, t_min = (0, 0.0)
    x_max, t_max = (L, 1.0)

    # Create tensors:
    t = np.linspace(t_min, t_max, num=t_dim).reshape(t_dim, 1)
    x = np.linspace(x_min, x_max, num=x_dim).reshape(x_dim, 1)
    usol = np.zeros((x_dim, t_dim)).reshape(x_dim, t_dim)

    # Obtain the value of the exact solution for each generated point:
    for i in range(x_dim):
        for j in range(t_dim):
            usol[i][j] = heat_eq_exact_solution(x[i], t[j])

    # Save solution:
    np.savez("heat_eq_data", x=x, t=t, usol=usol)


def gen_testdata():
    """Import and preprocess the dataset with the exact solution."""
    # Load the data:
    data = np.load("heat_eq_data.npz")
    # Obtain the values for t, x, and the excat solution:
    t, x, exact = data["t"], data["x"], data["usol"].T
    # Process the data and flatten it out (like labels and features):
    xx, tt = np.meshgrid(x, t)
    X = np.vstack((np.ravel(xx), np.ravel(tt))).T
    y = exact.flatten()[:, None]
    return X, y


# Problem parameters:
a = 0.4  # Thermal diffusivity
L = 1  # Length of the bar
n = 1  # Frequency of the sinusoidal initial conditions

# Generate a dataset with the exact solution (if you dont have one):
gen_exact_solution()

# The first argument to pde is 2-dimensional vector 
# where the first component(x[:,0]) is x-coordinate and the second componenet (x[:,1]) is the t-coordinate
# The second argument is the network output, i.e., the solution u(x,t)
# but here we use y as the name of the variable
def pde(x, y):
    """Expresses the PDE residual of the heat equation."""
    dy_t = dde.grad.jacobian(y, x, i=0, j=1)
    dy_xx = dde.grad.hessian(y, x, i=0, j=0)
    return dy_t - a * dy_xx


# Computational geometry:
geom = dde.geometry.Interval(0, L)
timedomain = dde.geometry.TimeDomain(0, 1)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

# Initial and boundary conditions:
bc = dde.icbc.DirichletBC(geomtime, lambda x: 0, lambda _, on_boundary: on_boundary)
ic = dde.icbc.IC(
    geomtime,
    lambda x: np.sin(n * np.pi * x[:, 0:1] / L),
    lambda _, on_initial: on_initial,
)

# Define the PDE problem and configurations of the network:
data = dde.data.TimePDE(
    geomtime,
    pde,
    [bc, ic],
    num_domain=2540,
    num_boundary=80,
    num_initial=160,
    num_test=2540,
)
net = dde.nn.FNN([2] + [20] * 3 + [1], "tanh", "Glorot normal")
model = dde.Model(data, net)

# Build and train the model:
model.compile("adam", lr=1e-3)
model.train(iterations=20000)
model.compile("L-BFGS")
losshistory, train_state = model.train()

# Plot/print the results
dde.saveplot(losshistory, train_state, issave=True, isplot=True)
X, y_true = gen_testdata()
y_pred = model.predict(X)
f = model.predict(X, operator=pde)
print("Mean residual:", np.mean(np.absolute(f)))
print("L2 relative error:", dde.metrics.l2_relative_error(y_true, y_pred))
np.savetxt("test.dat", np.hstack((X, y_true, y_pred)))